In [2]:
pip install opencv-python opencv-python-headless tensorflow numpy


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.8 MB ? eta -:--:--
    --------------------------------------- 0.5/38.8 MB 1.0 MB/s eta 0:00:37
    --------------------------------------- 0.8/38.8 MB 1.2 MB/s eta 0:00:33
   - -------------------------------------- 1.0/38.8 MB 1.1 MB/s eta 0:00:34
   - -------------------------------------- 1.3/38.8 MB 1.1 MB/s eta 0:00:35
   - -------------------------------------- 1.6/38.8 MB 1.1 MB/s eta 0:00:35
   - -------------------------------------- 1.6/38.8 MB 1.1 MB/s eta 0:00:35
   - -------------------------------------- 1.8/38.8 MB 1.1 MB/s eta 0:00:34
   -- ------------------------------------- 2.1/38.8 MB 1.1 MB/s eta 0:00:33
   -- ------------------------------------- 2.6/38.8 MB 1.2 MB/s eta 0:00:31
   -- ------------------------------------- 2.9/38.8 MB 1.2 MB/s eta 0:00:31
   -- --------------

In [5]:

import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to original dataset (update with your actual dataset directory)
dataset_path = r'C:\Users\GIGABYTE\Desktop\dataset'

# Path to save augmented images
augmented_images_dir = r'C:\Users\GIGABYTE\Desktop\new-dataset'
os.makedirs(augmented_images_dir, exist_ok=True)

# Lists to hold images and labels
images = []
labels = []

# Loop through all images in the dataset directory
for label in os.listdir(dataset_path):
    label_path = os.path.join(dataset_path, label)
    if os.path.isdir(label_path):  # Check if it's a directory (person folder)
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            img = cv2.imread(image_path)
            img = cv2.resize(img, (224, 224))  # Resize to a consistent size (224x224)

            images.append(img)
            labels.append(label)

# Convert images and labels to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Encode labels into integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Define the ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=30,           # Rotate images randomly by up to 30 degrees
    width_shift_range=0.2,       # Shift images horizontally by up to 20%
    height_shift_range=0.2,      # Shift images vertically by up to 20%
    shear_range=0.2,             # Shear images randomly
    zoom_range=0.2,              # Zoom into images randomly
    horizontal_flip=True,        # Randomly flip images horizontally
    fill_mode='nearest'          # Fill missing pixels after transformations
)

# Fit the generator to the images
datagen.fit(images)

# Save all augmented images
augmented_images_per_original = 5  # Number of augmented images per original image

for i in range(len(images)):  # Loop through each image in the dataset
    image = images[i:i+1]  # Get a single image from the batch (batch_size=1)
    label = labels[i]      # Get the label corresponding to this image

    # Create a subdirectory for each label/person in the augmented images directory
    label_dir = os.path.join(augmented_images_dir, str(label))
    os.makedirs(label_dir, exist_ok=True)

    # Create a generator for the augmented images for the current image
    augmented_images = datagen.flow(image, batch_size=1, save_to_dir=label_dir, 
                                    save_prefix='aug', save_format='jpg')

    # Generate and save multiple augmented images for each original image
    for j in range(augmented_images_per_original):  # Create augmented_images_per_original per image
        next(augmented_images)

print("All augmented images have been saved successfully.")


All augmented images have been saved successfully.


In [32]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
import pyttsx3 

# Initialize the TTS engine
engine = pyttsx3.init()

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the dataset
def load_images(dataset_path):
    images = []
    labels = []
    label_encoder = LabelEncoder()
    for folder_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, folder_name)
        if os.path.isdir(person_path):
            for filename in os.listdir(person_path):
                image_path = os.path.join(person_path, filename)
                image = cv2.imread(image_path)
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
                for (x, y, w, h) in faces:
                    face = gray[y:y + h, x:x + w]
                    face_resized = cv2.resize(face, (160, 160))
                    images.append(face_resized)
                    labels.append(folder_name)
    labels = label_encoder.fit_transform(labels)
    images = np.array(images)
    images = images.reshape(-1, 160, 160, 1) / 255.0  # Normalize
    return images, labels, label_encoder

# Dataset path
dataset_path = r'C:\Users\GIGABYTE\Desktop\new-dataset'  # Update this path
images, labels, label_encoder = load_images(dataset_path)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Build the CNN model
def build_model():
    model = models.Sequential([
        layers.InputLayer(input_shape=(160, 160, 1)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(len(np.unique(labels)), activation='softmax')  # Adjust for the number of classes
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model
model = build_model()
model.summary()
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=8)

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Save the trained model
model.save('face_recognition_model.keras')

# Example recognition function
def recognize_face(image_path, model, label_encoder):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        face = gray[y:y + h, x:x + w]
        face_resized = cv2.resize(face, (160, 160))
        face_resized = face_resized.reshape(1, 160, 160, 1) / 255.0
        prediction = model.predict(face_resized)
        label_index = np.argmax(prediction)
        label = label_encoder.inverse_transform([label_index])[0]
        
        # Display the name on the image
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Check if the recognized person is "person_chahd"
        if label == "person_chahd":
            cv2.putText(image, "Hello Chahd!", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            engine.say("Hello Chahd!")
            engine.runAndWait()
        if label == "person_ayoub":
            cv2.putText(image, "Hello Ayoub!", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            engine.say("Hello Ayoub!")
            engine.runAndWait()
        if label == "person_omar":
            cv2.putText(image, "Hello Omar!", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            engine.say("Hello Omar!")
            engine.runAndWait()
        if label == "person_jihen":
            cv2.putText(image, "Hello Jihen!", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            engine.say("Hello Jihen!")
            engine.runAndWait()

    # Resize the image to fit the window 
    image_resized = cv2.resize(image, (800, 600))  # Resize to smaller dimensions for display
    cv2.imshow("Recognized Face", image_resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test the model with a new image
recognize_face(r'C:\Users\GIGABYTE\Desktop\dataset\person_jihen\test-jihen3.jpg.jpg', model, label_encoder)


C:\Users\GIGABYTE\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_36 (Conv2D)              │ (None, 158, 158, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_36 (MaxPooling2D) │ (None, 79, 79, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 77, 77, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_37 (MaxPooling2D) │ (None, 38, 38, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 36, 36, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_38 (MaxPooling2D) │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 41472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 128)            │     5,308,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,401,732 (20.61 MB)

 Trainable params: 5,401,732 (20.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.4167 - loss: 1.2505 - val_accuracy: 0.6667 - val_loss: 0.9186
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.6302 - loss: 1.0730 - val_accuracy: 0.6667 - val_loss: 0.9696
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.4844 - loss: 1.2045 - val_accuracy: 0.6667 - val_loss: 1.0416
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.5312 - loss: 1.0688 - val_accuracy: 0.6667 - val_loss: 0.6794
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.5625 - loss: 0.9371 - val_accuracy: 0.8333 - val_loss: 0.6797
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.7708 - loss: 0.8361 - val_accuracy: 0.8333 - val_loss: 0.7360
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.8802 - loss: 0.5810 - val_accuracy: 0.6667 - val_loss: 0.5129
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.9271 - loss: 0.3594 - val_accuracy: 0.8333 - val_loss: 0.5094

In [44]:
model.save(r'C:\Users\GIGABYTE\Desktop\challenge ias\face.keras')

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model and label encoder
model = tf.keras.models.load_model(r'C:\Users\GIGABYTE\Desktop\challenge ias\face.keras')

# Capture video from webcam
cap = cv2.VideoCapture(0)  # 0 is usually the default camera
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def recognize_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = gray[y:y + h, x:x + w]
        face_resized = cv2.resize(face, (160, 160))
        face_resized = face_resized.reshape(1, 160, 160, 1) / 255.0

        prediction = model.predict(face_resized)
        label_index = np.argmax(prediction)
        confidence = np.max(prediction) * 100
        label = label_encoder.inverse_transform([label_index])[0]

        # Draw rectangle and display name & confidence
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} ({confidence:.2f}%)", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display personalized greetings if certain persons are detected
        if label == "person_chahd":
            cv2.putText(frame, "Hello Chahd!", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
        elif label == "person_ayoub":
            cv2.putText(frame, "Hello Ayoub!", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
        elif label == "person_omar":
            cv2.putText(frame, "Hello Omar!", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
        elif label == "person_jihen":
            cv2.putText(frame, "Hello Jihen!", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    return frame

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = recognize_faces(frame)
    cv2.imshow('Webcam Face Recognition', frame)

    # Press 'q' to quit the webcam feed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━